In [46]:
from tkinter import *
from PIL import ImageTk, Image
import threading
import cv2 


In [47]:
def hz_to_ms(hz):
    if hz == 0:
        raise ZeroDivisionError
    return int(round(1000/hz))

In [53]:
import tkinter as tk

class Controller:
    def __init__(self, root, box_1_hz, box_2_hz, stream_url = False):
        # Window stuff
        self.root = root
        self.root.title("Controller")
        root.attributes('-fullscreen', True)
        self.screen_width = self.root.winfo_screenwidth()
        self.screen_height = self.root.winfo_screenheight()
        
        # This is information about what camera you are using and the size of the video feed.
        if not stream_url:
            self.cap = cv2.VideoCapture(0)
        else:
            self.cap = cv2.VideoCapture(stream_url)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.screen_width // 3)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.screen_height // 3)
        

        # this sets up the GUI
        self.canvas = tk.Canvas(self.root, width=self.screen_width, height=self.screen_height)
        self.canvas.place(anchor='nw', x=0, y=0)
        self.video = tk.Label(self.root)
        self.video.place(anchor='center', relx=0.5, rely=0.5)
        
        self.down = self.create_flash_rect(self.screen_width // 2, self.screen_height // 2 + 450)
        self.up = self.create_flash_rect(self.screen_width // 2, self.screen_height // 2 - 450)
        # threading stuff
        # Allows for python to run tasks in parallel, with the tasks being the flashing. daemon=true basically kills the thread when you close the window.
        threading.Thread(target=self.flash, args=(self.up, hz_to_ms(box_1_hz)), daemon=True).start()
        threading.Thread(target=self.flash, args=(self.down, hz_to_ms(box_2_hz)), daemon=True).start()
        threading.Thread(target=self.open_camera).start()
        
    def run(self):
        self.root.mainloop()

    def open_camera(self):
        # Capture the video frame by frame
        if self.cap.isOpened():
            _, frame = self.cap.read()
            # Resize Image and convert Colors
            frame = cv2.cvtColor(cv2.resize(frame, (self.screen_width//3, self.screen_height//3)), cv2.COLOR_BGR2RGBA)
            # Capture the latest frame and transform to image
            self.photo_image = ImageTk.PhotoImage(image=Image.fromarray(frame))
            # Displaying camera feed onto the gui
            self.video.configure(image=self.photo_image)
        self.root.after(10, self.open_camera)
    
    def create_flash_rect(self, x, y, size=65):
        # Create a flashing rectangle centered at (x, y).
        rect = self.canvas.create_rectangle(
            x - size, y - size, x + size, y + size, fill='black'
        )
        return rect

    def flash(self, rect, interval):
        # Toggle the fill color of a rectangle at a specified interval
        current_fill = self.canvas.itemcget(rect, 'fill')
        self.canvas.itemconfig(rect, fill='white' if current_fill == 'black' else 'black')
        self.root.after(interval, lambda: self.flash(rect, interval))
        

# Initializes the Code
window = tk.Tk()
RC_Controller = Controller(window, 12, 15)
RC_Controller.run()
